# Natural Language Processing with Disaster Tweets
# 1. Introduction
In this project, we tackle the Kaggle competition "Natural Language Processing with Disaster Tweets." The challenge is to build a model that can predict whether a given tweet is about a real disaster (target = 1) or not (target = 0). This task is essential in real-world applications for quickly identifying urgent information during disasters.

## Dataset Overview

* Training Set: Contains 7,613 tweets with labels.
* Test Set: Contains 3,263 tweets without labels (used for submission).
* Features:
    * id: Unique identifier for each tweet.
    * text: The tweet content.
    * location: The location the tweet was sent from (may be null).
    * keyword: A particular keyword from the tweet (may be null).
    * target: 1 if the tweet is about a real disaster, 0 otherwise (only in training data).
# 2. Exploratory Data Analysis (EDA)
## Loading Libraries and Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

## Data Inspection

In [ ]:
print("Training Data Shape:", train.shape)
print("Test Data Shape:", test.shape)
print("Unique target values:", train['target'].unique())
print("Value counts:\n", train['target'].value_counts())
train.head()

## Missing Values

In [ ]:
train.isnull().sum()

Observations:
* location and keyword have missing values.
* text and target are complete.

## Target Distribution

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(x='target', data=train)
plt.title('Distribution of Disaster vs. Non-Disaster Tweets')
plt.xlabel('Target')
plt.ylabel('Count')
plt.show()

Observation: The classes are slightly imbalanced.

## Word Cloud Visualization

In [ ]:
from wordcloud import WordCloud

# Disaster tweets
disaster_tweets = train[train['target'] == 1]['text'].values
non_disaster_tweets = train[train['target'] == 0]['text'].values

# Generate word clouds
def generate_wordcloud(text, title):
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(' '.join(text))
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(title)
    plt.show()

generate_wordcloud(disaster_tweets, 'Disaster Tweets')
generate_wordcloud(non_disaster_tweets, 'Non-Disaster Tweets')

## Data Cleaning Plan

Based on EDA, we plan to:

* Clean the text data by:
    * Removing URLs, HTML tags, and mentions.
    * Handling contractions (e.g., "don't" to "do not").
    * Removing punctuation and special characters.
    * Converting text to lowercase.
* Tokenization: Split text into tokens.
* Word Embedding: Use Word2Vec embeddings to represent words in vectors.

# 3. Data Preprocessing
## Text Cleaning Function

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)         # Remove HTML tags
    text = re.sub(r'@[A-Za-z0-9_]+', '', text) # Remove mentions
    text = re.sub(r'[^a-zA-Z]', ' ', text)    # Remove punctuation
    text = text.split()
    text = [word for word in text if word not in stop_words]
    text = ' '.join(text)
    return text

train['clean_text'] = train['text'].apply(clean_text)
test['clean_text'] = test['text'].apply(clean_text)

## Tokenization and Sequencing

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the tokenizer
vocab_size = 10000  # You can adjust this value
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(train['clean_text'])

# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(train['clean_text'])
X_test_sequences = tokenizer.texts_to_sequences(test['clean_text'])

# Pad sequences to ensure equal length
max_length = 100  # You can adjust this based on the data
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

# Target variable
y_train = train['target'].values

## Word Embedding with Word2Vec

We will use pre-trained Word2Vec embeddings to represent words.

* Word2Vec: An embedding method that maps words into vector space, capturing semantic meaning.

In [ ]:
# import gensim.downloader as api

# word2vec = api.load('word2vec-google-news-300')

# embedding_dim = 300
vocab_size = len(tokenizer.word_index) + 1

# # Create embedding matrix
# embedding_matrix = np.zeros((vocab_size, embedding_dim))
# for word, i in tokenizer.word_index.items():
#     if word in word2vec:
#         embedding_matrix[i] = word2vec[word]

# 4. Model Architectures
We will build and compare three different RNN models:

* Model 1: LSTM
* Model 2: GRU
* Model 3: Bidirectional LSTM

## Common Model Components

* Embedding Layer: Transforms integer representations into dense vectors of fixed size.
* RNN Layer: Captures sequential information.
* Dense Layer: Outputs the final prediction.

## Model 1: LSTM

An LSTM (Long Short-Term Memory) network is a type of RNN capable of learning long-term dependencies. It mitigates the vanishing gradient problem, making it suitable for text data where context from earlier words can be important.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model_lstm = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

## Model 2: GRU

A GRU (Gated Recurrent Unit) is a gating mechanism in RNNs that, like LSTM, helps to solve the vanishing gradient problem. It has fewer parameters than LSTM, which can lead to faster training times.


In [ ]:
from tensorflow.keras.layers import GRU

model_gru = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length),
    GRU(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

model_gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru.summary()

## Model 3: Bidirectional LSTM

A Bidirectional LSTM processes the sequence both forward and backward, capturing information from past and future states. This can be beneficial when the context after a word is important for understanding.


In [ ]:
from tensorflow.keras.layers import Bidirectional

model_bi_lstm = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length),
    Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)),
    Dense(1, activation='sigmoid')
])

model_bi_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bi_lstm.summary()

# 5. Training and Evaluation

## Splitting the Data

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
X_train, X_val, y_train_split, y_val = train_test_split(X_train_padded, y_train, test_size=0.2, random_state=42)

## Training the Models

Hyperparameters
* Batch Size: 64
* Epochs: 5

In [ ]:
#Model 1: LSTM
history_lstm = model_lstm.fit(X_train, y_train_split, epochs=5, batch_size=64, validation_data=(X_val, y_val))

#Model 2: GRU
history_gru = model_gru.fit(X_train, y_train_split, epochs=5, batch_size=64, validation_data=(X_val, y_val))

#Model 3: Bidirectional LSTM
history_bi_lstm = model_bi_lstm.fit(X_train, y_train_split, epochs=5, batch_size=64, validation_data=(X_val, y_val))

## Training Analysis

Observation: The model shows improvement over epochs.

# 6. Evaluation and Results

Model 1: LSTM

In [ ]:
val_loss_lstm, val_acc_lstm = model_lstm.evaluate(X_val, y_val)

# Plot accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_lstm.history['accuracy'], label='Train Accuracy')
plt.plot(history_lstm.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model 1: LSTM Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history_lstm.history['loss'], label='Train Loss')
plt.plot(history_lstm.history['val_loss'], label='Validation Loss')
plt.title('Model 1: LSTM Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

print(f"LSTM Validation Accuracy: {val_acc_lstm:.4f}")

Model 2: GRU

In [ ]:
val_loss_gru, val_acc_gru = model_gru.evaluate(X_val, y_val)

# Plot accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_gru.history['accuracy'], label='Train Accuracy')
plt.plot(history_gru.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model 2: GRU Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history_gru.history['loss'], label='Train Loss')
plt.plot(history_gru.history['val_loss'], label='Validation Loss')
plt.title('Model 2: GRU Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

print(f"GRU Validation Accuracy: {val_acc_gru:.4f}")

Model 3: Bidirectional LSTM

In [ ]:
val_loss_bi_lstm, val_acc_bi_lstm = model_bi_lstm.evaluate(X_val, y_val)

# Plot accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_bi_lstm.history['accuracy'], label='Train Accuracy')
plt.plot(history_bi_lstm.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model 3: Bidirectional LSTM Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history_bi_lstm.history['loss'], label='Train Loss')
plt.plot(history_bi_lstm.history['val_loss'], label='Validation Loss')
plt.title('Model 3: Bidirectional LSTM Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

print(f"Bidirectional LSTM Validation Accuracy: {val_acc_bi_lstm:.4f}")

Comparing the Models

In [ ]:
results = pd.DataFrame({
    'Model': ['LSTM', 'GRU', 'Bidirectional LSTM'],
    'Validation Accuracy': [val_acc_lstm, val_acc_gru, val_acc_bi_lstm],
    'Validation Loss': [val_loss_lstm, val_loss_gru, val_loss_bi_lstm]
})

results

## Comparing the Models Visually

We can also plot the validation accuracy of all models in a single plot for direct comparison.

In [ ]:
# Plot validation accuracy of all models
plt.figure(figsize=(8, 6))
plt.plot(history_lstm.history['val_accuracy'], label='LSTM')
plt.plot(history_gru.history['val_accuracy'], label='GRU')
plt.plot(history_bi_lstm.history['val_accuracy'], label='Bidirectional LSTM')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.show()

Interpretation:

* This plot allows us to see which model consistently performs better across epochs.
* The model with the highest validation accuracy curve is likely the best performer.
* Similarly, we can plot the validation loss:

In [ ]:
# Plot validation loss of all models
plt.figure(figsize=(8, 6))
plt.plot(history_lstm.history['val_loss'], label='LSTM')
plt.plot(history_gru.history['val_loss'], label='GRU')
plt.plot(history_bi_lstm.history['val_loss'], label='Bidirectional LSTM')
plt.title('Validation Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.legend()
plt.show()

Interpretation:

* Lower validation loss indicates better model performance.
* Comparing the curves helps identify which model converges faster and generalizes better.
* Observation: Training and validation accuracy increase over epochs; loss decreases.

## Predictions on Test Data

In [ ]:
predictions = model_bi_lstm.predict(X_test_padded)
predictions = (predictions > 0.5).astype(int).reshape(-1)

## Submission

In [ ]:
# Create a submission DataFrame
submission = pd.DataFrame({'id': test['id'], 'target': predictions})
submission.to_csv('submission.csv', index=False)

# Display the first few rows
submission.head()

# 7. Conclusion
In this project, we built an LSTM-based neural network to classify disaster-related tweets. Key takeaways include:

* Data Cleaning significantly impacts model performance.
* Word Embeddings like Word2Vec help in capturing semantic meaning.
* LSTM Networks are effective for text classification tasks.
* Hyperparameter Tuning and Regularization (Dropout) improve model generalization.

Future Improvements

* Experiment with GRU or Bidirectional LSTM for potential performance gains.
* Use More Advanced Embeddings like GloVe or BERT.
* Hyperparameter Optimization using tools like GridSearchCV or Bayesian Optimization.

References
* Kaggle Competition: [Natural Language Processing with Disaster Tweets](https://www.kaggle.com/c/nlp-getting-started)
* TensorFlow Keras Documentation: [Recurrent Layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Recurrent)
* Word2Vec: [Google's Word2Vec Model](https://code.google.com/archive/p/word2vec/)